# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nantucket,41.2835,-70.0995,27.03,87,0,4.12,US,1694624687
1,1,port-aux-francais,-49.3500,70.2167,0.36,67,100,16.10,TF,1694624913
2,2,longyearbyen,78.2186,15.6401,1.91,75,75,7.72,SJ,1694624914
3,3,hammerfest,70.6634,23.6821,6.25,87,75,7.72,NO,1694624872
4,4,utrik,11.2278,169.8474,28.30,77,22,6.69,MH,1694624914


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale =1,
    color = "City",
    alpha = 0.8,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

#Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21) &(city_data_df["Wind Speed"]<4.5) &(city_data_df["Cloudiness"]==0)]


# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,hawaiian paradise park,19.5933,-154.9731,25.50,86,0,0.00,US,1694624917
48,48,holualoa,19.6228,-155.9522,24.37,69,0,2.68,US,1694624924
182,182,fuengirola,36.5400,-4.6247,26.58,69,0,4.12,ES,1694624963
210,210,semnan,35.5729,53.3971,24.86,36,0,4.26,IR,1694624987
340,340,kailua-kona,19.6406,-155.9956,26.44,72,0,2.68,US,1694625030
369,369,hilo,19.7297,-155.0900,25.62,83,0,0.45,US,1694625014
454,454,gargalianoi,37.0667,21.6333,24.64,65,0,2.97,GR,1694625062
471,471,samsat,37.5819,38.4742,24.62,17,0,2.06,TR,1694625067
511,511,zahedan,29.4963,60.8629,25.29,19,0,1.46,IR,1694625077
527,527,kalaleh,37.3807,55.4916,22.26,62,0,1.07,IR,1694625081


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country","Lat", "Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

/var/folders/gj/_3jlnbzx3f72qvfy2r_dmt5h0000gp/T/ipykernel_28169/3894578519.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Lat,Lng,Humidity,Hotel Name
18,hawaiian paradise park,US,19.5933,-154.9731,86,
48,holualoa,US,19.6228,-155.9522,69,
182,fuengirola,ES,36.5400,-4.6247,69,
210,semnan,IR,35.5729,53.3971,36,
340,kailua-kona,US,19.6406,-155.9956,72,
369,hilo,US,19.7297,-155.0900,83,
454,gargalianoi,GR,37.0667,21.6333,65,
471,samsat,TR,37.5819,38.4742,17,
511,zahedan,IR,29.4963,60.8629,19,
527,kalaleh,IR,37.3807,55.4916,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [30]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row ["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
fuengirola - nearest hotel: Hotel Casa Consistorial
semnan - nearest hotel: مرکز آموزشی رفاهی فرهنگیان - خانه معلم
kailua-kona - nearest hotel: Kona Seaside Hotel
hilo - nearest hotel: Dolphin Bay Hotel
gargalianoi - nearest hotel: Messinian Horizons
samsat - nearest hotel: No hotel found
zahedan - nearest hotel: هتل صالح;بانک ملی
kalaleh - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
shache - nearest hotel: 莎車新盛賓舘


,City,Country,Lat,Lng,Humidity,Hotel Name
18,hawaiian paradise park,US,19.5933,-154.9731,86,No hotel found
48,holualoa,US,19.6228,-155.9522,69,Kona Hotel
182,fuengirola,ES,36.5400,-4.6247,69,Hotel Casa Consistorial
210,semnan,IR,35.5729,53.3971,36,مرکز آموزشی رفاهی فرهنگیان - خانه معلم
340,kailua-kona,US,19.6406,-155.9956,72,Kona Seaside Hotel
369,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
454,gargalianoi,GR,37.0667,21.6333,65,Messinian Horizons
471,samsat,TR,37.5819,38.4742,17,No hotel found
511,zahedan,IR,29.4963,60.8629,19,هتل صالح;بانک ملی
527,kalaleh,IR,37.3807,55.4916,62,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name","Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)